# Softwaretechnologien für Natürlichsprachliche System
---
Sommersemester 2019

Modulverantwortliche: Prof. Dr. Udo Hahn, Sven Büchel

Lehrstuhl für Computerlingustik 

Friedrich-Schiller-Universität Jena



**Blatt 4 vom 20. Juni 2019: Datenanalyse**

Abgabe bis Dienstag, den **25. Juni 2019**, 23:59

Per Email an **johannes.hellrich@uni-jena.de**

---


**Vorname:** ...

** Name:** ...

** Matrikelnummer:** ...

---



**Erreichte Punktzahl: .../10**

-----

## Aufgabe 1: Textprozessierung mit NLTK
**(... /2 Punkten)**

Das Beispiel zeigt, wie ein getaggtes Korpus mit NLTK geladen und eine Statistik zu den darin enthaltenen Wortarten erstellt werden kann. In der Aufgabe soll das gleiche Korpus ohne Annotationen mit einem in NLTK enthaltenen Tagger POS-annotiert werden. Analysieren Sie das Ergebnis und vergleichen Sie es mit den kanonischen Annotationen aus dem Beispiel, welche Unterschiede fallen Ihnen bei den Wörtern am Anfang und bei den häufigsten POS auf?

In [25]:
import nltk
import collections

nltk.download('brown') # Installiert Brown Korpus
tagged_corpus = nltk.corpus.brown.tagged_words()
print(tagged_corpus[:10]) 

#Statistiken zum Korpus
counter = collections.Counter()
for word, pos in tagged_corpus:
  counter[pos] += 1
sorted_counts = counter.most_common(10)
print(sorted_counts)



[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN')]
[('NN', 152470), ('IN', 120557), ('AT', 97959), ('JJ', 64028), ('.', 60638), (',', 58156), ('NNS', 55110), ('CC', 37718), ('RB', 36464), ('NP', 34476), ('VB', 33693), ('VBN', 29186), ('VBD', 26167), ('CS', 22143), ('PPS', 18253), ('VBG', 17893), ('PP$', 16872), ('TO', 14918), ('PPSS', 13802), ('CD', 13510)]


###Lösung

In [0]:
nltk.download('averaged_perceptron_tagger') #Installiert den NLTK default Tagger
untagged_corpus = nltk.corpus.brown.words()
#todo, vgl. http://www.nltk.org/book/ch05.html

#Klassifikation mit NLTK


## Korpus

In [0]:
import nltk
nltk.download('names') #installiert ein "corpus" aus Namen mit Geschlechtsangabe
print("Dateien = Labels:",nltk.corpus.names.fileids())
print("Inhalt:",nltk.corpus.names.words()[:5])
ng_list = [(name,gender[:1])
           for gender in names.fileids()
           for name in names.words(gender)]   
print("Paare aus (Namen, Label):",ng_list[:5])

print("Weibliche Namen:",sum([1 if x[1]=="f" else 0 for x in ng_list]))
print("Männliche Namen:",sum([1 if x[1]=="m" else 0 for x in ng_list]))

##Randomisiertes Trainingsmaterial

In [0]:
import random
random.shuffle(ng_list)

test_data = ng_list[:1000]
train_data = ng_list[1000:]

print("Testdaten:",test_data[:5])
print("Trainingsdaten:",train_data[:5])

##Feature Definition

In [0]:
def gender_features(word):
    features = {}
    word = word.lower()
    features['first_letter'] = word[0]
    return features

train_set = nltk.classify.apply_features(gender_features, train_data)
test_set = nltk.classify.apply_features(gender_features, test_data)

##Training

In [0]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print("Accuracy:",nltk.classify.accuracy(classifier, test_set)) #ca. 0.63

##Fehleranalyse

In [0]:
classifier.show_most_informative_features(5)  #beste Features

In [0]:
def getErrors(test_set_,classifier, limit=20):
    errors = []
    for (name, tag) in test_set_:
        guess = classifier.classify(gender_features(name))
        if guess != tag:
            errors.append( (tag, guess, name) )
    for (tag, guess, name) in sorted(errors)[:limit]:
        print('correct=%-8s guess=%-8s name=%-30s' % (tag, guess, name))

In [0]:
getErrors(test_data,classifier) #falsch klassifiziert

## Aufgabe 2: Bessere Features
**(... /2 Punkten)**

Erweitern Sie den angegebenen Code durch das hinzufügen weiterer Features um die Genauigkeit des Klassifikators zu erhöhen. Sie sollten Werte von 0.75 oder mehr erreichen können. Beachten Sie dabei auch die Möglichkeiten zur Anzeige der informativsten Features bzw. falsch klassifizierter Wörter.

###Lösung

In [0]:
def your_gender_features(word):
    features = {}
    word = word.lower()
    features['first_letter'] = word[0]
    # more features here ...
    return features

#Features für Daten extrahieren
train_set = nltk.classify.apply_features(your_gender_features, train_data)
test_set = nltk.classify.apply_features(your_gender_features, test_data)

classifier = nltk.NaiveBayesClassifier.train(train_set)
print("Ihre Accuracy:",nltk.classify.accuracy(classifier, test_set)) # > 0.75?
classifier.show_most_informative_features(5) 

---

## Aufgabe 3: Kreuzvalidierung

**(... / 3 Punkten)**

Die Genauigkeit eines Klassifikators ist Abhängig von der genauen Verteilung der Daten im Trainings- und Testmaterial. Um verlässliche Aussagen über die Qualität eines Klassifikators machen zu können, ist es üblich Experimente zu wiederholen. Bei der Kreuzvalidierung wird ein Datenset in n Teilezerlegt, von denen reihum jedes als Testdatensatz fungiert, während die übrigen als Trainingsdaten verwendet werden. Ergänzen Sie den unten angegebenen Code, um eine 5-fache Kreuzvalidierung des Classifiers durchzuführen und ermitteln Sie so seine mittlere Performance und deren Standardabweichung (etwa über die in NumPy eingebauten Funktionen).

###Vorbereitung

In [0]:
import random
import nltk
import numpy as np

nltk.download('names') #installiert ein "Korpus" aus Namen mit Geschlechtsangabe
ng_list = [(name,gender[:1])
           for gender in names.fileids()
           for name in names.words(gender)] 
random.shuffle(ng_list)

def gender_features(word):
    features = {}
    word = word.lower()
    features['first_letter'] = word[0]
    return features
  
#7.5k Wörter mit Label als Daten für den folgenden Vergleich
all_data = nltk.classify.apply_features(gender_features, ng_list[:7500])
  

### Lösung


In [0]:
def experiment(train, test):
  #todo
  return accuracy

def get_folds(data, n):
  size = len(data) // n # // für Ganzzahl
  current = 0
  while current < len(data):
    #todo
    yield test_split, train_splits
  
accuracies = []
for train, test in get_folds(all_data, 5):
  accuracies.append(experiment(train, test))
  
#todo
print("Accuracy:")
print("Standardabweichung:")

---

#Klassifikation mit sklearn

Dieser Abschnitt behandelt sklearn am Beispiel Dokumentenklassifikation.

##Korpus laden

In [2]:
import sklearn
from sklearn.datasets import fetch_20newsgroups
texts = fetch_20newsgroups(subset='train',
                          
                             shuffle=True, random_state=42)

print("Textbeispiele:")
print("\n".join(texts.data[0].split("\n")[:3]))
print("-"*5)
print("\n".join(texts.data[1].split("\n")[:3]))
print("-"*20)
print("Alle Labels:",texts.target)
print("Label Namen:",texts.target_names)



Textbeispiele:
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
-----
From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
--------------------
Alle Labels: [7 4 4 ... 3 1 8]
Label Namen: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


##Bag of Word Repräsentation

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
texts_bow = count_vect.fit_transform(texts.data)
index = count_vect.vocabulary_.get(u'car') 
print("Car in 1. Dokument:",texts_bow[0,index]) 
print("Car in 2. Dokument:",texts_bow[1,index])

Car in 1. Dokument: 5
Car in 2. Dokument: 0


##Beispielsklassifikator

In [4]:
from sklearn.naive_bayes import MultinomialNB
import numpy as np

#Training
classifier = MultinomialNB().fit(texts_bow, texts.target)

#Reklassifikation
reclassification = classifier.predict(texts_bow)
print("Genauigkeit bei Reklassifikation:",
      np.mean(reclassification == texts.target))

#Klassifikation neuer Dokumente
docs_new = ['Drugs are bad, mmmkay?', 'I want a new and fast GPU!']
X_new_counts = count_vect.transform(docs_new)
predicted = classifier.predict(X_new_counts)
print("Label der neuen Dokumente:",
      ", ".join([texts.target_names[label] for label in predicted]))

Genauigkeit bei Reklassifikation: 0.9245182959165635
Label der neuen Dokumente: talk.politics.misc, sci.crypt


##Evaluation


In [5]:
test_texts = fetch_20newsgroups(subset='test',shuffle=True, random_state=42)

test_bow = count_vect.transform(test_texts.data)
predicted = classifier.predict(test_bow)
np.mean(predicted == test_texts.target) 



0.7728359001593202

## Aufgabe 4: Klassifikatorenvergleich

**(... / 3 Punkten)**

Testen Sie weitere Klassifikatoren in sklearn (mindestens die unten angegebenen) in Hinblick auf ihre Genauigkeit bei der Dokumentenklassifikation. Nutzen Sie dazu die in sklearn enthaltene Kreuzvalidierungsmöglichkeit, nutzen Sie alle Dokumente des Datensatzes und verwenden Sie eine Funktion / Schleife für die Evaluation der Modelle (also kein Copy&Paste Code!). Welcher Klassifikator ist am genauesten?


In [0]:
import sklearn.neighbors
from sklearn.model_selection import cross_val_score

to_test=[sklearn.svm.LinearSVC(max_iter=5), 
         sklearn.naive_bayes.MultinomialNB(), 
         sklearn.linear_model.LogisticRegression(max_iter=5),
         sklearn.neighbors.KNeighborsClassifier()]



### Lösung:


In [0]:
#alle Texte laden und vorverarbeiten statt nur train/test, siehe https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html
all_texts_bow #todo 
all_texts_labels #todo

#Kreuzvalidierung einzelner Modelle, Funktion/Schleife erstellen
scores = cross_val_score(model_to_test, all_texts_bow, all_texts_labels, cv=5) #5 splits
print(scores.mean() , scores.std())

---